<a href="https://colab.research.google.com/github/pradeeplaxkar/Heart-Attack/blob/main/Final_Real_Time_Face_Authorization_(ArcFace_%2B_RetinaFace)_%E2%80%94_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ======================
# STEP 0: Install deps
# ======================
!pip install -q opencv-python opencv-contrib-python deepface tensorflow gradio

import os, cv2, time, wave, struct, glob
import numpy as np
from numpy.linalg import norm
from deepface import DeepFace
import gradio as gr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.9 MB/s eta 0:00:00
25-09-15 07:05:27 - Directory /root/.deepface has been created
25-09-15 07:05:27 - Directory /root/.deepface/weights has been created


In [ ]:
# ======================
# STEP 1: Paths & Setup
# ======================
DB_ROOT = "authorized_faces"   # <-- Put one subfolder per person: authorized_faces/Alice/*.jpg
os.makedirs(DB_ROOT, exist_ok=True)

# Helpful: show instructions
print("""
Upload enrollment images like:

authorized_faces/
  Alice/
    1.jpg
    2.jpg
  Bob/
    selfie1.png
    office2.jpg

Then run the next cell to build the database.
""")


Upload enrollment images like:

authorized_faces/
  Alice/
    1.jpg
    2.jpg
  Bob/
    selfie1.png
    office2.jpg

Then run the next cell to build the database.



In [ ]:
# ======================
# STEP 2: Utilities
# ======================
def l2_normalize(v):
    v = np.asarray(v, dtype=np.float32)
    return v / (norm(v) + 1e-12)

def euclidean(a, b):
    return norm(a - b)

def variance_of_laplacian(img_gray):
    # focus measure (higher = sharper)
    return cv2.Laplacian(img_gray, cv2.CV_64F).var()

def mean_brightness(img_gray):
    return float(np.mean(img_gray))

def ensure_uint8(img):
    """DeepFace.extract_faces returns aligned 'face' in [0,1] float.
       Convert safely to uint8 if needed."""
    arr = img
    if arr.dtype != np.uint8:
        arr = np.clip(arr, 0.0, 1.0) * 255.0
        arr = arr.astype("uint8")
    return arr


In [ ]:
# ======================
# STEP 3: Build Database
# ======================
# We compute embeddings with:
#   - Detection/Alignment: RetinaFace (only at enrollment)
#   - Embedding model: ArcFace
# We then compute per-person centroid and adaptive threshold.

EMBED_MODEL = "ArcFace"      # SOTA
DET_BACKEND = "retinaface"   # robust detector

class PersonEntry:
    def __init__(self, name):
        self.name = name
        self.embeddings = []   # list of l2-normalized embeddings
        self.centroid = None
        self.threshold = 0.60  # fallback; will be adapted

    def finalize(self):
        if len(self.embeddings) == 0:
            return
        # centroid of normalized embeddings, then re-normalize
        C = np.mean(np.stack(self.embeddings, axis=0), axis=0)
        self.centroid = l2_normalize(C)

        # adaptive threshold: 90th percentile of in-class distances + margin
        dists = [euclidean(e, self.centroid) for e in self.embeddings]
        if len(dists) >= 3:
            p90 = float(np.percentile(dists, 90))
            self.threshold = max(0.35, min(0.80, p90 + 0.05))  # clamp to sensible range
        elif len(dists) == 2:
            self.threshold = max(0.45, min(0.75, np.mean(dists) + 0.05))
        else:
            # single image: keep conservative default
            self.threshold = 0.60

authorized_db = []  # list[PersonEntry]

def build_database(db_root=DB_ROOT):
    authorized_db.clear()

    # detect if user used subfolders (recommended)
    subfolders = [p for p in glob.glob(os.path.join(db_root, "*")) if os.path.isdir(p)]
    used_subfolders = len(subfolders) > 0

    if not used_subfolders:
        print("⚠ No subfolders found. Using each image in root as a separate identity by its filename stem.")
        image_paths = [p for p in glob.glob(os.path.join(db_root, "*")) if os.path.isfile(p)]
        # group by stem (before dot)
        groups = {}
        for p in image_paths:
            name = os.path.splitext(os.path.basename(p))[0]
            groups.setdefault(name, []).append(p)
        name_to_images = groups.items()
    else:
        name_to_images = []
        for person_dir in subfolders:
            person = os.path.basename(person_dir)
            imgs = [p for p in glob.glob(os.path.join(person_dir, "*")) if os.path.isfile(p)]
            if imgs:
                name_to_images.append((person, imgs))

    n_people = 0
    for name, imgs in name_to_images:
        entry = PersonEntry(name)
        for img_path in imgs:
            try:
                # represent does detection+alignment here ONCE at enrollment
                rep = DeepFace.represent(
                    img_path,
                    model_name=EMBED_MODEL,
                    detector_backend=DET_BACKEND,
                    enforce_detection=True
                )
                # DeepFace.represent returns a list of detections
                for item in rep:
                    emb = l2_normalize(np.array(item["embedding"], dtype=np.float32))
                    entry.embeddings.append(emb)
                print(f"✅ {name}: +{len(rep)} face(s) from {os.path.basename(img_path)}")
            except Exception as e:
                print(f"❌ {name} - {os.path.basename(img_path)}: {e}")

        if len(entry.embeddings) > 0:
            entry.finalize()
            print(f"   ↳ centroid set; adaptive threshold={entry.threshold:.3f}; samples={len(entry.embeddings)}")
            authorized_db.append(entry)
            n_people += 1

    print(f"\n📚 Database built: {n_people} identities.")
    if n_people == 0:
        print("➡️ Add images to authorized_faces/<NAME>/ and run this cell again.")

# Run this after uploading your images
build_database()


25-09-15 07:09:19 - 🔗 arcface_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/arcface_weights.h5 to /root/.deepface/weights/arcface_weights.h5...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/arcface_weights.h5
To: /root/.deepface/weights/arcface_weights.h5
100%|██████████| 137M/137M [00:00<00:00, 207MB/s]


25-09-15 07:09:23 - retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5
100%|██████████| 119M/119M [00:00<00:00, 240MB/s] 


✅ Pradeep: +1 face(s) from WIN_20250914_14_07_03_Pro.jpg
✅ Pradeep: +1 face(s) from WIN_20250914_14_07_06_Pro.jpg
✅ Pradeep: +1 face(s) from WIN_20250914_14_07_09_Pro.jpg
✅ Pradeep: +1 face(s) from WIN_20250914_14_07_07_Pro.jpg
✅ Pradeep: +1 face(s) from WIN_20250914_14_07_00_Pro.jpg
✅ Pradeep: +1 face(s) from WIN_20250914_14_06_59_Pro (2).jpg
✅ Pradeep: +1 face(s) from WIN_20250914_14_07_08_Pro.jpg
✅ Pradeep: +1 face(s) from WIN_20250914_14_07_04_Pro.jpg
✅ Pradeep: +1 face(s) from WIN_20250914_14_06_55_Pro.jpg
✅ Pradeep: +1 face(s) from WIN_20250914_14_07_05_Pro.jpg
✅ Pradeep: +1 face(s) from WIN_20250914_14_07_01_Pro.jpg
✅ Pradeep: +1 face(s) from WIN_20250914_14_06_57_Pro.jpg
✅ Pradeep: +2 face(s) from WIN_20250914_14_07_02_Pro.jpg
✅ Pradeep: +1 face(s) from WIN_20250914_14_06_59_Pro.jpg
   ↳ centroid set; adaptive threshold=0.800; samples=15
✅ Registrar: +1 face(s) from WIN_20250915_10_25_07_Pro.jpg
✅ Registrar: +1 face(s) from WIN_20250915_10_25_04_Pro (2).jpg
✅ Registrar: +1 face

In [ ]:
# ======================
# STEP 5: Live Inference
# ======================
# Important: we use DeepFace.extract_faces to get ALIGNED chips: f["face"]
# Then we embed with detector_backend="skip" to avoid re-detection.
# Add quality gates + temporal smoothing

MIN_FACE = 60            # min height/width of face chip in px
MIN_BRIGHT = 20          # mean grayscale lower bound
MAX_BRIGHT = 235         # mean grayscale upper bound
MIN_SHARPNESS = 20.0     # variance of Laplacian threshold
VOTE_N = 3               # require 3 consecutive "authorized" frames per face to confirm
FRAME_COOLDOWN = 0.75    # seconds between beeps

last_beep_ts = 0.0
consec_auth = 0  # simple stream-wide vote; good enough for single-subject demos

def classify_embedding(emb):
    """Return (is_authorized, best_name, best_dist, best_thresh) using per-person centroid+threshold."""
    if not authorized_db:
        return False, None, 9e9, 0.0
    emb = l2_normalize(np.array(emb, dtype=np.float32))
    best_name, best_dist, best_thresh = None, 9e9, 0.0
    for person in authorized_db:
        d = euclidean(emb, person.centroid)
        if d < best_dist:
            best_dist = d
            best_name = person.name
            best_thresh = person.threshold
    return (best_dist < best_thresh), best_name, float(best_dist), float(best_thresh)

def live_check(frame_rgb):
    global last_beep_ts, consec_auth

    # Convert to BGR for OpenCV ops
    frame_bgr = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

    try:
        detections = DeepFace.extract_faces(
            frame_bgr,
            detector_backend=DET_BACKEND,
            enforce_detection=False
        )
    except Exception:
        detections = []

    any_unauth = False
    any_face = False

    for f in detections or []:
        fa = f.get("facial_area", {})
        x, y, w, h = int(fa.get("x", 0)), int(fa.get("y", 0)), int(fa.get("w", 0)), int(fa.get("h", 0))
        x2, y2 = x + w, y + h
        if w <= 0 or h <= 0:
            continue

        # Use ALIGNED face chip from DeepFace
        chip = f.get("face", None)
        if chip is None:
            # fallback to raw crop (less reliable)
            chip = frame_bgr[max(0,y):max(0,y2), max(0,x):max(0,x2)]
        chip = ensure_uint8(chip)

        # Quality gates
        #if chip.shape[0] < MIN_FACE or chip.shape[1] < MIN_FACE:
            #label = "Face too small — move closer"
            #color = (255, 165, 0)  # orange
            #cv2.rectangle(frame_bgr, (x, y), (x2, y2), color, 2)
            #cv2.putText(frame_bgr, label, (x, max(0, y-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            #continue

        gray = cv2.cvtColor(chip, cv2.COLOR_BGR2GRAY)
        bright = mean_brightness(gray)
        sharp = variance_of_laplacian(gray)
        #if not (MIN_BRIGHT <= bright <= MAX_BRIGHT) or sharp < MIN_SHARPNESS:
            #label = "Low quality — add light / hold steady"
            #color = (255, 165, 0)
            #cv2.rectangle(frame_bgr, (x, y), (x2, y2), color, 2)
            #cv2.putText(frame_bgr, label, (x, max(0, y-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            #continue

        any_face = True

        # Embed WITHOUT re-detecting: detector_backend="skip"
        try:
            rep = DeepFace.represent(
                chip,
                model_name=EMBED_MODEL,
                detector_backend="skip",   # <— critical
                enforce_detection=False
            )
            if not rep or not isinstance(rep, list):
                continue
            emb = rep[0]["embedding"]
        except Exception:
            continue

        ok, name, dist, thr = classify_embedding(emb)

        if ok:
            consec_auth = min(VOTE_N, consec_auth + 1)
            confirmed = (consec_auth >= VOTE_N)
            color = (0, 200, 0) if confirmed else (0, 180, 120)
            label = f"Authorized: {name}  d={dist:.2f} < {thr:.2f}" if confirmed else f"Checking… {name} d={dist:.2f}"
        else:
            consec_auth = 0
            any_unauth = True
            color = (0, 0, 255)
            label = f"Unauthorized  d={dist:.2f} ≥ {thr:.2f}"

        cv2.rectangle(frame_bgr, (x, y), (x2, y2), color, 2)
        cv2.putText(frame_bgr, label, (x, max(0, y-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.75, color, 2)

    # Decide beep + status
    if any_face and not any_unauth and consec_auth >= VOTE_N:
        status = "✅ Authorized"
        beep = None
    elif any_unauth:
        status = "⚠ Unauthorized detected!"
        # throttle beep a little to avoid constant noise
        now = time.time()
        beep = "alert.wav" if (now - last_beep_ts) >= FRAME_COOLDOWN else None
        if beep: last_beep_ts = now
    else:
        status = "🙂 Show your face to the camera"
        beep = None
        consec_auth = 0

    return cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB), beep, status


In [ ]:
# ======================
# STEP 6: Gradio UI
# ======================
with gr.Blocks() as demo:
    gr.Markdown("## 👁️ Real-Time Face Authorization (ArcFace + RetinaFace) — Colab\n"
                "Aligned chips, adaptive thresholds, quality gates, temporal smoothing.")
    with gr.Row():
        cam = gr.Image(sources="webcam", streaming=True, type="numpy", label="Webcam")
        out = gr.Image(type="numpy", label="Detections")
    beep_audio = gr.Audio(label="Beep", autoplay=True)
    status = gr.Label(label="Status")
    cam.stream(live_check, inputs=cam, outputs=[out, beep_audio, status])

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7480ede62062480ef6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
